<a href="https://colab.research.google.com/github/athiniotim/Final_Essay_Forest_Fires/blob/main/forest_fires_kathimerini_articles_scraper_Michaela_Athinioti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):

  headers = {
        "User-Agent": "Mozilla/5.0"
    }

  response = requests.get(url, headers=headers)


  if response.status_code != 200:
    #print(f"Error: {response.status_code}")
    return None
  else:

    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):

  article_block = soup.find("article")
  # article url
  article_url = article_url
  # title
  try:
    title = article_block.find("h1").text.strip()
  except:
    try:
      title = soup.find("div", {"class": "container is-fullhd p-3"}).h1.text.strip()
    except:
      title=""
  # datetime
  try:
    datetime_str = article_block.find("time")["datetime"]
  except:
    try:
      datetime_str = soup.find("div", {"class": "container is-fullhd p-3"}).find("time")["datetime"]
    except:
      datetime_str = ""
      date_str = ""
      time_str = ""
  if datetime_str != "":
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("+")[0]
  # section
  try:
    section = article_block.find("span", class_=lambda c: c and "nx-single-category-title" in c).text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("span", class_=lambda c: c and "author" in c).text.strip()
  except:
    try:
      author = soup.find("div", {"class": "container is-fullhd p-3"}).find("a", {"rel": "author"}).text.strip()
    except:
      author = ""
  # excerpt
  try:
    excerpt = article_block.find("div", {"class": "nx-excerpt pb-5"}).text.strip()
  except:
    try:
      excerpt = soup.find("div", {"class": "container is-fullhd p-3"}).find("div", {"class": "newsletter-description nx-excerpt my-5"}).text.strip()
    except:
      excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", {"id": "main-content"}).find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_link = article_block.find("div", {"class": "post-thumbnail"}).img["src"]
  except:
    feat_img_link=""
  # feature image caption
  try:
    image_caption = article_block.find("div", {"class": "img-caption mb-5"}).text.strip()
  except:
    image_caption=""


  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "text_body": text_body, "feat_img_link": feat_img_link, "feat_img_cation": image_caption}

  return articleD

In [ ]:
def scrape_kathimerini_articles_pages(teasers_df):

  all_articles_dataL = []

  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):

    article_url = row['article_url']

    soup = get_webpage_soup(article_url)

    articleD = get_article_elements_from_soup(soup, article_url)

    all_articles_dataL.append(articleD)

    delay = random.uniform(1, 3)
    time.sleep(delay)

  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Δοκιμή του scraper**

In [ ]:
teasers_df = pd.read_csv("/content/drive/MyDrive/Final Essay ΕΔΔΕ.2 Αθηνιώτη Μιχαέλα/clean csv/forest_fires_kathimerini_teaser_csv.csv")
teasers_df.head()

,title,article_url,date,time,section,excerpt,author
0,"ΟΗΕ: Το 2024, η κλιματική αλλαγή προκάλεσε ακρ...",https://www.kathimerini.gr/world/563396869/oie...,2024-12-30,NaN,ΚΟΣΜΟΣ,Σύμφωνα με τον Παγκόσμιο Μετεωρολογικό Οργανισ...,NaN
1,"«Ζούμε σε μια νέα, επικίνδυνη εποχή» – Η υπερθ...",https://www.kathimerini.gr/life/environment/56...,2024-12-27,NaN,ΠΕΡΙΒΑΛΛΟΝ,"Το 2024, η πιο ζεστή χρονιά που έχει καταγραφε...",NaN
2,Ερευνα: Οι 14 μεγάλες πυρκαγιές του 2024,https://www.kathimerini.gr/life/environment/56...,2024-12-17,NaN,ΠΕΡΙΒΑΛΛΟΝ,Στην έκθεση του Αστεροσκοπείου και της WWF Ελλ...,NaN
3,Πορτογαλία και Ισπανία έζησαν τον πιο ζεστό Νο...,https://www.kathimerini.gr/world/563358376/por...,2024-12-05,NaN,ΚΟΣΜΟΣ,Η μέση θερμοκρασία του Νοεμβρίου στην Πορτογαλ...,NaN
4,Δύο νέες πλατφόρμες για την ανάλυση του κλίματος,https://www.kathimerini.gr/society/563356729/d...,2024-12-05,NaN,ΚΟΙΝΩΝΙΑ,Δύο νέους ψηφιακούς κόμβους ανάλυσης της ατμόσ...,NaN


In [ ]:
forest_fires_articles_kathimerini_df = scrape_kathimerini_articles_pages(teasers_df)
forest_fires_articles_kathimerini_df

Processing rows: 100%|██████████| 648/648 [31:34<00:00,  2.92s/it]


Scraping completed. Fetshed 648 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.kathimerini.gr/world/563396869/oie...,"ΟΗΕ: Το 2024, η κλιματική αλλαγή προκάλεσε ακρ...",2024-12-30,23:42:00,Κόσμος,Newsroom,Σύμφωνα με τον Παγκόσμιο Μετεωρολογικό Οργανισ...,Η κλιματική αλλαγή προκάλεσε ακραίες μετεωρολο...,https://www.kathimerini.gr/wp-content/uploads/...,Καλιφόρνια: Προειδοποίηση στους κατοίκους να μ...
1,https://www.kathimerini.gr/life/environment/56...,,,,,,,Από το μικρό γαλλικό αρχιπέλαγος της Μαγιότ έω...,,
2,https://www.kathimerini.gr/life/environment/56...,,,,,,,"Δύσκολες κλιματολογικές συνθήκες, νέες προκλήσ...",,
3,https://www.kathimerini.gr/world/563358376/por...,Πορτογαλία και Ισπανία έζησαν τον πιο ζεστό Νο...,2024-12-05,22:11:00,Κόσμος,Newsroom,Η μέση θερμοκρασία του Νοεμβρίου στην Πορτογαλ...,Υψηλότατες για την εποχή θερμοκρασίες καταγράφ...,https://www.kathimerini.gr/wp-content/uploads/...,Shutterstock
4,https://www.kathimerini.gr/society/563356729/d...,Δύο νέες πλατφόρμες για την ανάλυση του κλίματος,2024-12-05,23:34:00,Κοινωνία,Γιάννης Ελαφρός,Δύο νέους ψηφιακούς κόμβους ανάλυσης της ατμόσ...,"Δύο νέα ισχυρά εργαλεία καταγραφής, ανάλυσης κ...",https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. INTIME/ ΧΑΛΚΙΟΠΟΥΛΟΣ ΝΙΚΟΣ.
...,...,...,...,...,...,...,...,...,...,...
643,https://www.kathimerini.gr/world/562221718/kal...,Καλιφόρνια: Τουλάχιστον 14 νεκροί από τη σφοδρ...,2023-01-10,17:27:40,Κόσμος,Newsroom,«Τα θύματα είναι περισσότερα από εκείνα που σκ...,Σφοδρή βροχόπτωση αναμένεται να πλήξει και πάλ...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. REUTERS/ Erica Urech
644,https://www.kathimerini.gr/world/562221523/cop...,Copernicus: Το 2022 ήταν το δεύτερο πιο ζεστό ...,2023-01-10,15:53:30,Κόσμος,Newsroom,«Ζούμε ήδη τις καταστροφικές συνέπειες του θερ...,Το 2022 ήταν το δεύτερο πιο ζεστό έτος που έχε...,https://www.kathimerini.gr/wp-content/uploads/...,Φωτ. AP/ Michael Probst
645,https://www.kathimerini.gr/world/562221112/vra...,Βραζιλία: Οι ρίζες της εξέγερσης των μπολσοναρ...,2023-01-10,12:24:33,Κόσμος,Δημήτρης Αθηνάκης,Τα σχέδια του προέδρου Λούλα για την προστασία...,"Στα πρώτα σχόλιά του μετά την επίθεση, ο νέος ...",https://www.kathimerini.gr/wp-content/uploads/...,Προεκλογική πινακίδα υπέρ του πρώην προέδρου τ...
646,https://www.kathimerini.gr/world/562215640/kal...,"Καλιφόρνια – Κακοκαιρία: Ένα νεκρό παιδί, χιλι...",2023-01-05,18:02:52,Κόσμος,Newsroom,Περισσότερες από 180.000 κατοικίες και επιχειρ...,Ένα παιδί σκοτώθηκε και δεκάδες χιλιάδες άνθρω...,https://www.kathimerini.gr/wp-content/uploads/...,REUTERS/Carlos Barria


In [ ]:
forest_fires_articles_kathimerini_df.to_csv("/content/drive/MyDrive/Final Essay ΕΔΔΕ.2 Αθηνιώτη Μιχαέλα/clean csv/forest_fires_kathimerini_articles_csv", index=False)